In [ ]:
# Make sure that Python starts in Workflow-folder or else the modules will be screewed up!
import sys, os, getpass
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

user = getpass.getuser()

if user == "sidsel":
    parquet_path = "/home/sidsel/workspace/sparkdata/parquet"
elif user == "svanhmic":
    parquet_path = "/home/svanhmic/workspace/data/DABAI/sparkdata/parquet"
    
# Start the logger.
import logging
logger_tester = logging.getLogger(__name__)
logger_tester.setLevel(logging.INFO)
logger_file_handler_param = logging.FileHandler('/tmp/workflow_test.log')
logger_formatter_param = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(message)s')

logger_tester.addHandler(logger_file_handler_param)
logger_file_handler_param.setFormatter(logger_formatter_param)

In [ ]:
from shared.Extension_to_timeit import pretty_time_result
from shared.GeneralDataImport import GeneralDataImport

dataIO = GeneralDataImport(parquet_path+"/normal_cluster_n_1000.parquet")

In [ ]:
dataIO.select_columns()

In [ ]:
df = dataIO.data_frame

df.count()

In [ ]:
#import data!
#from pyspark.sql import functions as F
#from shared.create_dummy_data import create_dummy_data 

#test_timer = %timeit -o feature_data = create_dummy_data(1000, "x y z", "label", outlier_number=0.2, outlier_factor=20)
#feature_data = feature_data.select([(10*F.col(i)).alias(i) for i in ["x","y","z"]])
#feature_data.orderBy('x',ascending=[0,0,0]).show()

In [ ]:
#Select parameters
#from cleaning.CreateParametersCleaning import ParamsCleaning

#params = ParamsCleaning()
#parameters = params.select_parameters()

In [ ]:
#parameters

In [ ]:
#print(params.output_parameters(parameters))
test_params_1 = {'tol': 0.001, 'k': 3, 'maxIter': 10, 'algorithm': 'GaussianMixture', 'seed': 1080866016001745000}

In [ ]:
print(sc.defaultMinPartitions)
print(sc.defaultParallelism)
conf = sc.getConf()
#conf.getAll()

In [ ]:
from cleaning.ExecuteCleaningWorkflow import ExecuteWorkflow

n = [3, 5] # number of samples 1000, 100000, 1000000

execution_model = ExecuteWorkflow(
    dict_params=test_params_1,
    cols_features=list(map(lambda x: x.name,dataIO.list_features)), 
    cols_labels=list(map(lambda x: x.name,dataIO.list_label)))

collection_of_data = []
collection_of_model = []
collection_of_transformed = []
counts = [i.rdd.getNumPartitions() for i in collection_of_data]
counts
collection_of_data.append(df)

In [ ]:
for idx, data in enumerate(collection_of_data):
    model_timer = %timeit -o collection_of_model.append(execution_model.execute_pipeline(data)) 
    transformer_timer = %timeit -o execution_model.apply_model(collection_of_model[0],data)
    collection_of_model = collection_of_model[:idx+1]
    logger_tester.info('Iteration '+str(idx)+' for training model : '+pretty_time_result(model_timer))
    logger_tester.info('Iteration '+str(idx)+' for transforming model : '+pretty_time_result(transformer_timer))
    #merged_df.write.parquet('/home/svanhmic/workspace/data/DABAI/sparkdata/parquet/merged_df_parquet')

In [ ]:
collection_of_model

In [ ]:
df_result = execution_model.apply_model(collection_of_model[0],collection_of_data[0])
df_result.show(3)

In [ ]:
from cleaning.ShowCleaning import ShowResults

results = ShowResults(execution_model.params_labs_feats, list_features=['a','b'], list_labels=['id','k','dimension'])

In [ ]:
df_with_dists = results.select_prototypes(df_result)

In [ ]:
execution_model.params_labs_feats

In [ ]:
pdf = df_result.select('prediction','centers','cov').distinct().toPandas()


In [ ]:
for idx, pred in pdf.iterrows():
    print(pred['prediction'], pred['cov'].toArray(), pred['centers'])

In [ ]:
dsm=pdf[pdf['prediction']==1]

In [ ]:
dsm['cov'].tolist()

In [ ]:
from shared.create_dummy_data import create_normal_cluster_data_spark
import pandas as pd

In [ ]:
plot_gaussians(df_result,['a','b'])

# Waring experimental comming up!!!

In [ ]:
sb.palplot(sb.hls_palette(len(pdf_mean), l=.3, s=.8))
plt.show()

sb.hls_palette(len(pdf_mean), l=.3, s=.8)

In [ ]:
ax = sb.jointplot(data=pd_results,x='x',y='y')
plt.show()

In [ ]:
from shared import ComputeDistances
from pyspark.sql import types as T
from pyspark.ml.linalg import VectorUDT, Vectors

udf_subtract = F.udf(lambda x, y: Vectors.dense(ComputeDistances.subtract_vectors(x,y)),VectorUDT())
udf_picewise_dist = F.udf(lambda x,y: Vectors.dense(np.abs(x.array-y.array)),VectorUDT())

df = runTest.withColumn("picewise_distance",udf_picewise_dist(F.col("centers"),F.col("features")))

In [ ]:
df.show(5)

In [ ]:
testDf = (df
          .select('prediction','picewise_distance')
          .rdd
          .reduceByKey(lambda x,y : x+y)
          #.mapValues(lambda x: x.array.tolist())
          .map(lambda x: itertools.chain.from_iterable(([x[0]],x[1])))
          .map(lambda x: [float(i) for i in x])
          .toDF(["prediction", 'picewise_distance_x','picewise_distance_y','picewise_distance_z'])
         )

testDf.take(5)

In [ ]:
pdf = testDf.toPandas()
pdf['total'] = pdf['picewise_distance_x']+pdf['picewise_distance_y']


sb.set_style("white")
sb.set_context({"figure.figsize": (24, 10)})

sb.barplot(x='prediction',y=pdf['total'],data=pdf, color = "blue")
bottom_plot = sb.barplot(x='prediction',y=pdf['picewise_distance_x'],data=pdf, color = "red")

topbar = plt.Rectangle((0,0),1,1,fc="red", edgecolor = 'none')
bottombar = plt.Rectangle((0,0),1,1,fc='blue',  edgecolor = 'none')
l = plt.legend([bottombar, topbar], ['Bottom Bar', 'Top Bar'], loc=1, ncol = 2, prop={'size':16})
l.draw_frame(False)

#Optional code - Make plot look nicer
sb.despine(left=True)


plt.xlabel('Clusters')
plt.ylabel('Diff')
plt.show()

In [ ]:
prediction_pdf = pdf['prediction']
cumulative_pdf = pdf.loc[:, 'picewise_distance_x':'picewise_distance_z'].cumsum(axis=1)